In [ ]:
!pip3 install pickle5
import pickle5 as pickle
import pandas as pd
import re
import csv

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)
root_dir = "/content/gdrive/My Drive/masters_thesis/"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
def unpickle_data(filename):
    with open(filename, "rb") as fh:
      data = pickle.load(fh)
      return data
    #return unpickled_df
notes_file = root_dir + "notes_full.pickle"
admissions_file = root_dir + "admissions.pickle"
# notes_df = unpickle_data(notes_file)
# admissions_df = unpickle_data(admissions_file)
# notes_df.to_csv(root_dir + "notes.csv", index=False)
# admissions_df.to_csv(root_dir + "admissions.csv", index=False)
#!cp "notes.csv" base_dir
#!cp "admissions.csv" base_dir
#print(unpickle_data(admissions_file).head())

In [ ]:
def read_data(filename):
  df = pd.read_csv(filename)
  print(df.head(1))
  print(df.dtypes)
  print(df.index)
  print(df.columns)
  return df

notes_file = root_dir + "notes.csv"
#admissions_file = root_dir + "admissions.csv"
notes_df = read_data(notes_file)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  # This is added back by InteractiveShellApp.init_path()


   row_id  subject_id   hadm_id   chartdate charttime storetime  \
0       1       23224  174680.0  2147-12-05       NaN       NaN   

            category description  cgid iserror  \
0  Discharge summary      Report   NaN           

                                                text  
0  Admission Date:  [**2823-9-29**]              ...  
row_id           int64
subject_id       int64
hadm_id        float64
chartdate       object
charttime       object
storetime       object
category        object
description     object
cgid           float64
iserror         object
text            object
dtype: object
RangeIndex(start=0, stop=2078705, step=1)
Index(['row_id', 'subject_id', 'hadm_id', 'chartdate', 'charttime',
       'storetime', 'category', 'description', 'cgid', 'iserror', 'text'],
      dtype='object')


In [ ]:
now_df = read_data(root_dir + "setup6_training.csv")
print(len(now_df))

    hadm_id                                             source  \
0  133857.0  69 year old male who on fishing trip alone tod...   

                                              target  
0  HPI: This is a 69 year old male who is primari...  
hadm_id    float64
source      object
target      object
dtype: object
RangeIndex(start=0, stop=5981, step=1)
Index(['hadm_id', 'source', 'target'], dtype='object')
5981


In [ ]:
notes_df = notes_df[notes_df["iserror"] == " "]

In [ ]:
def notes_info(notes_df):
  #print("notes size", len(notes_df))
  #print("CATEGORY EXAMPLE")
  #print(len(notes_df["category"].unique()))
  #print("DESCRIPTION EXAMPLE")
  #desc_size = notes_df["description"].unique()
  #print("description size", len(desc_size))
  #for i in range(len(desc_size)):
    #print(desc_size[i])
  #print(notes_df["description"].unique())
  # print("TEXT EXAMPLE")
  # print(notes_df["text"][0])
  discharge_summary_df = notes_df.loc[notes_df["category"] == "Nursing"]
  print(len(discharge_summary_df))
  for i in range(1):
    print(discharge_summary_df.iloc[i]["text"])

def get_examples(notes_df, text_file):
  cat_examples = []
  with open(text_file, 'w') as txtfile:
    categories = notes_df["category"].unique()
    for c in categories:
      c_str = "Category: " + c + "\n"
      part_df = notes_df.loc[notes_df["category"] == c]
      txtfile.write(c_str)
      for i in range(5):
        ex_str = "Example " + str(i + 1) + ":\n" 
        txtfile.write(ex_str)
        txtfile.write(part_df.iloc[i]["text"])



def discharge_summary_info(notes_df):
  discharge_summary_df = notes_df.loc[notes_df["category"] == "Discharge summary"]
  print("Length of discharge df", len(discharge_summary_df))
  sections_to_count = {}
  sections_to_length = {}
  hadm_to_final_divides = {}
  for i in range(len(discharge_summary_df)):
    hadm_id = discharge_summary_df.iloc[i]["hadm_id"]
    text = discharge_summary_df.iloc[i]["text"]
    sections = re.split('([a-zA-Z]+):\n|([a-zA-Z]+\s[a-zA-Z]+):\n', text)
    section = ""
    final_divides = {}
    for i in range(1,len(sections)):
      if sections[i] != None:
        if section != "":
          final_divides[section] = sections[i]
          section = ""
        else:
          section = sections[i]
    for key, value in final_divides.items():
      sentence_len = len(value.split("."))
      if key in sections_to_count:
        curr_counter = sections_to_count[key]
        total_length = sections_to_length[key]
        curr_counter += 1
        total_length += sentence_len
        sections_to_count[key] = curr_counter
        sections_to_length[key] = total_length
      else:
        sections_to_count[key] = 1
        sections_to_length[key] = sentence_len
    hadm_to_final_divides[hadm_id] = final_divides
  return sections_to_count, sections_to_length, hadm_to_final_divides

def nursing_info(notes_df):
  nursing_df = notes_df.loc[notes_df["category"] == "Nursing"]
  print(len(nursing_df))
  for i in range(1):
    text = nursing_df.iloc[i]["text"]
    sections = re.split('(\w+:)', text)

def grouping_by_patients(notes_df):
  filter_df = notes_df[notes_df.iserror != '1']
  #print(len(filter_df))
  #categories = ["Discharge summary", "Nursing"]
  #specific_rows = filter_df[filter_df.category.isin(categories)]
  #print(len(specific_rows)) #there are 278359 that are either discharge summary or nursing
  # no_error_rows = specific_rows[specific_rows.iserror == 'false']
  #print(len(no_error_rows))
  hadm_to_discharge = {}
  hadm_to_nursing = {}
  for index, row in filter_df.iterrows():
    hadm_id = row['hadm_id']
    curr_category = row["category"]
    if (curr_category == "Nursing"):
      curr_date = row["chartdate"]
      curr_text = row["text"]
      if hadm_id in hadm_to_nursing:
        curr_nursing = hadm_to_nursing[hadm_id]
        curr_nursing.append((curr_date,curr_text))
        hadm_to_nursing[hadm_id] = curr_nursing
      else:
        hadm_to_nursing[hadm_id] = [(curr_date,curr_text)]
    if (curr_category == "Discharge summary"):
      curr_date = row["chartdate"]
      curr_text = row["text"]
      if hadm_id in hadm_to_discharge:
        curr_discharge = hadm_to_discharge[hadm_id]
        curr_discharge.append((curr_date,curr_text))
        hadm_to_discharge[hadm_id] = curr_discharge
      else:
        hadm_to_discharge[hadm_id] = [(curr_date,curr_text)]
  return hadm_to_discharge, hadm_to_nursing


# https://mimic.mit.edu/docs/iii/tables/noteevents/
#notes_file = root_dir + "notes.csv"
#admissions_file = root_dir + "admissions.csv"
#notes_df = read_data(notes_file)
#text_file = root_dir + "category_examples.txt"
#get_examples(notes_df, text_file)

#nursing_info(notes_df)
hadm_to_discharge, hadm_to_nursing = grouping_by_patients(notes_df)
sections_to_count, sections_to_length, hadm_to_final_divides = discharge_summary_info(notes_df)

count_filepath = root_dir  + "updated_sections_to_count.csv"
len_filepath = root_dir  + "updated_sections_to_length.csv"
avg_filepath = root_dir  + "updated_sections_to_avg_length.csv"
hadm_discharge_file_path = root_dir  + "hadm_to_discharge.csv"
with open(count_filepath, 'w') as f:
  for key in sections_to_count.keys():
    f.write("%s, %s\n" % (key, sections_to_count[key]))

with open(len_filepath, 'w') as f:
  for key in sections_to_length.keys():
    f.write("%s, %s\n" % (key, sections_to_length[key]))

sections_to_avg_length = {}
for key, val in sections_to_count.items():
  total_length = sections_to_length[key]
  avg_length = float(total_length) / float(val)
  sections_to_avg_length[key] = avg_length

with open(avg_filepath, 'w') as f:
  f.write("%s, %s\n" % ("Section", "Avg_Num_Sentences"))
  for key in sections_to_avg_length.keys():
    f.write("%s, %s\n" % (key, sections_to_avg_length[key]))

# with open(hadm_discharge_file_path, 'w') as f:
#   f.write("%s, %s\n" % ("hadm_id", "discharge_summary"))
#   for key in hadm_to_discharge.keys():
#     f.write("%s, %s\n" % (key, hadm_to_discharge[key][-1][1]))

list_of_hadm_ids = []
for key in hadm_to_discharge.keys():
  list_of_hadm_ids.append(key)

# with open(hadm_discharge_file_path, 'w') as csvfile:
#     header_key = ['hadm_id', 'discharge_summary']
#     new_val = csv.DictWriter(csvfile, fieldnames=header_key)
#     new_val.writeheader()
#     for key in hadm_to_discharge.keys():
#       list_of_hadm_ids.append(key)
#       new_val.writerow({'hadm_id': key, 'discharge_summary': hadm_to_discharge[key][-1][1]})


#read_data(admissions_file)

Length of discharge df 55177


In [ ]:
def summarize_text(paragraph):
  counter = 0
  max_counter = 3
  summary_text = ""
  sentences = paragraph.split(".")
  for i in sentences:
    if (counter < max_counter):
      curr_sentence = i + "."
      summary_text += curr_sentence
      counter += 1
    else:
      break
  return summary_text

def get_final_summary(section_dict):
  summaries = []
  for key,value in section_dict.items():
    summary = summarize_text(value)
    summary_lines = summary.split(".")
    summaries.append(summary)
  return summaries

def get_dict_str(section_dict):
  text = ""
  for key,value in section_dict.items():
    text += key + ": " + value + " "
  return text

In [ ]:
#final_discharge_df = pd.read_csv(hadm_discharge_file_path)
def create_setup1(hadm_to_nursing, file_to_write):
  fieldnames = ["hadm_id", "source", "target", "discharge_summary"]
  with open(file_to_write, 'w') as csvfile:
    csvwriter = csv.DictWriter(csvfile, delimiter=',', fieldnames=fieldnames)
    csvwriter.writerow(dict((fn,fn) for fn in fieldnames))
    #writer.writerow(["Source", "Target"])
    test_array = []
    counter = 0
    print("length of list of hadm ids", list_of_hadm_ids)
    for row in list_of_hadm_ids:
      if (counter == 9999):
        break
      hadm_id = row
      if (hadm_id in hadm_to_nursing):
        curr_nursing = hadm_to_nursing[hadm_id]
        total_nursing = ""
        for n in curr_nursing:
          total_nursing += "\n" + n[1]
        test_array.append({"source": total_nursing, "target": hadm_to_discharge[hadm_id][-1][1], "discharge_summary": hadm_to_discharge[hadm_id][-1][1], "hadm_id": hadm_id})
        counter += 1
          #writer.writerow({sources[t], targets[t]})
    for row in test_array:
      csvwriter.writerow(row)

file_name = root_dir + "setup1_nursing_discharge.csv"
#create_setup1(hadm_to_nursing, file_name)


In [ ]:
# import re  
import nltk  
# import string
# import heapq
!pip install sumy
from sumy.nlp.stemmers import Stemmer
import pandas
nltk.download('punkt')
nltk.download('stopwords')
from sumy.utils import get_stop_words
from sumy.nlp.stemmers import Stemmer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer as sumytoken

from sumy.summarizers.luhn import LuhnSummarizer


def luhn_summarizer(text, stemmer, LANGUAGE, SENTENCES_COUNT):
    parser = PlaintextParser.from_string(text, sumytoken(LANGUAGE))
    summarizer_luhn = LuhnSummarizer(stemmer)
    stop_words = get_stop_words(LANGUAGE)
    complete_stop_words = set(["Assessment", "Response", "Plan", "Action"])
    for i in stop_words:
      complete_stop_words.add(i)
    summarizer_luhn.stop_words = complete_stop_words
    sentences = []
    for sentence in summarizer_luhn(parser.document, SENTENCES_COUNT):
        a = sentence
        sentences.append(str(a))
    return " ".join(sentences)

LANGUAGE = "english"
SENTENCES_COUNT = 3
stemmer = Stemmer(LANGUAGE)

import csv

modified_setup2_filepath = root_dir + "modified_setup2_luhn_discharge.csv"


def create_modified_setup2(modified_setup2_filepath):
  with open(modified_setup2_filepath, 'w') as csvfile:
    fieldnames = ["hadm_id", "source", "target", "discharge_summary"]
    csvwriter = csv.DictWriter(csvfile, delimiter=',', fieldnames=fieldnames)
    csvwriter.writerow(dict((fn,fn) for fn in fieldnames))
    counter = 0
    test_array = []
    for row in list_of_hadm_ids:
      if (counter == 9999):
        break
      hadm_id = row
      if (hadm_id in hadm_to_nursing):
        curr_nursing = hadm_to_nursing[hadm_id]
        total_nursing = ""
        if len(curr_nursing) == 1:
          total_nursing = curr_nursing[0][1]
        elif len(curr_nursing) >= 2:
          total_nursing = curr_nursing[0][1]
          total_nursing = "\n" + curr_nursing[-1][1]
        target_source = hadm_to_discharge[hadm_id][-1][1]
        luhn_summary = luhn_summarizer(target_source, stemmer, LANGUAGE, SENTENCES_COUNT)
        test_array.append({"source": total_nursing, "target": luhn_summary, "discharge_summary": target_source, "hadm_id": hadm_id})
        counter += 1
    for row in test_array:
      csvwriter.writerow(row)

#create_modified_setup2(modified_setup2_filepath)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
setup3_filepath = root_dir  + "setup3_sentence_3_summary.csv"
modified_setup3_filepath = root_dir  + "modified_setup3_sentence_3_summary.csv"

def create_setup3(setup3_filepath):
  with open(setup3_filepath, 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["hadm_id", "source", "target", "discharge_summary"])
    counter = 0
    for row in list_of_hadm_ids:
      if (counter == 9999):
        break
      hadm_id = row
      source = hadm_to_final_divides[hadm_id]
      summary_3_per_sec = get_final_summary(source)
      source_text = get_dict_str(source)
      summary_text = ".".join(summary_3_per_sec)
      discharge_summary = hadm_to_discharge[hadm_id][-1][1]
      writer.writerow([hadm_id, source_text, summary_text, discharge_summary])
      counter += 1

def create_modified_setup3(modified_setup3_filepath):
  with open(modified_setup3_filepath, 'w') as csvfile:
    fieldnames = ["hadm_id", "source", "target", "discharge_summary"]
    csvwriter = csv.DictWriter(csvfile, delimiter=',', fieldnames=fieldnames)
    csvwriter.writerow(dict((fn,fn) for fn in fieldnames))
    counter = 0
    test_array = []
    for row in list_of_hadm_ids:
      if (counter == 9999):
        break
      hadm_id = row
      if (hadm_id in hadm_to_nursing):
        curr_nursing = hadm_to_nursing[hadm_id]
        total_nursing = ""
        if len(curr_nursing) == 1:
          total_nursing = curr_nursing[0][1]
        elif len(curr_nursing) >= 2:
          total_nursing = curr_nursing[0][1]
          total_nursing = "\n" + curr_nursing[-1][1]
        target_source = hadm_to_final_divides[hadm_id]
        summary_3_per_sec = get_final_summary(target_source)
        summary_text = ".".join(summary_3_per_sec)
        test_array.append({"source": total_nursing, "target": summary_text, "discharge_summary": hadm_to_discharge[hadm_id][-1][1], "hadm_id": hadm_id})
        counter += 1
    for row in test_array:
      csvwriter.writerow(row)

#create_setup3(setup3_filepath)
#create_modified_setup3(modified_setup3_filepath)

In [ ]:
setup4_filepath = root_dir  + "setup4_nursing_hist_section.csv"

def create_setup4(setup4_filepath):
  with open(setup4_filepath, 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["hadm_id", "source", "target", "discharge_summary"])
    counter = 0
    print(len(hadm_to_nursing))
    for row in list_of_hadm_ids:
      if (counter == 9999):
        break
      hadm_id = row
      source = hadm_to_final_divides[hadm_id]
      if hadm_id in hadm_to_nursing:
        curr_nursing = hadm_to_nursing[hadm_id][-1][1]
        if "Present Illness" in source:
          hist_present = source["Present Illness"]
          writer.writerow([hadm_id, curr_nursing, hist_present, hist_present])
          counter += 1

#create_setup4(setup4_filepath)

In [ ]:
# another setup
# source: each nursing note in between _NNEWNOTE
# target: both sections in between _NNEWNOTE
# another setup -> modified setup 1
# put everything...not just 10,000
#entity or topic based extraction
def create_setup5(setup5_filepath):
  fieldnames = ["hadm_id", "source", "target", "discharge_summary"]
  with open(setup5_filepath, 'w') as csvfile:
    csvwriter = csv.DictWriter(csvfile, delimiter=',', fieldnames=fieldnames)
    csvwriter.writerow(dict((fn,fn) for fn in fieldnames))
    #writer.writerow(["Source", "Target"])
    test_array = []
    print("length of list of hadm ids", list_of_hadm_ids)
    for row in list_of_hadm_ids:
      hadm_id = row
      if (hadm_id in hadm_to_nursing):
        curr_nursing = hadm_to_nursing[hadm_id]
        total_nursing = curr_nursing[0][1]
        for n in range(1, len(curr_nursing)):
          total_nursing += "\n_NNEWNOTE\n" + curr_nursing[n][1]
        test_array.append({"source": total_nursing, "target": hadm_to_discharge[hadm_id][-1][1], "discharge_summary": hadm_to_discharge[hadm_id][-1][1], "hadm_id": hadm_id})
          #writer.writerow({sources[t], targets[t]})
    for row in test_array:
      csvwriter.writerow(row)

setup5_filepath = root_dir + "setup5_nursing_discharge.csv"
#create_setup5(setup5_filepath)

In [ ]:
def create_setup6(setup6_filepath):
  fieldnames = ["hadm_id", "source", "target", "discharge_summary"]
  with open(setup6_filepath, 'w') as csvfile:
    csvwriter = csv.DictWriter(csvfile, delimiter=',', fieldnames=fieldnames)
    csvwriter.writerow(dict((fn,fn) for fn in fieldnames))
    #writer.writerow(["Source", "Target"])
    test_array = []
    for row in list_of_hadm_ids:
      hadm_id = row
      if (hadm_id in hadm_to_nursing):
        curr_nursing = hadm_to_nursing[hadm_id]
        total_nursing = curr_nursing[0][1]
        for n in range(1, len(curr_nursing)):
          total_nursing += "\n_NNEWNOTE\n" + curr_nursing[n][1]
        source = hadm_to_final_divides[hadm_id]
        if "Present Illness" in source and "Discharge Instructions" in source:
          hist_present = source["Present Illness"]
          instr_present = source["Discharge Instructions"]
          target = hist_present + "\n_NNEWNOTE\n" + instr_present
          test_array.append({"source": total_nursing, "target": target, "discharge_summary": target, "hadm_id": hadm_id})
          #writer.writerow({sources[t], targets[t]})
    for row in test_array:
      csvwriter.writerow(row)

setup6_filepath = root_dir + "setup6_nursing_hist_discharge.csv"
#create_setup6(setup6_filepath)